
# [Meillionen](https://github.com/openmodelingfoundation/meillionen): Model Interfaces and Coupling

## [Open Modeling Foundation](https://openmodelingfounation.github.io)

Calvin Pritchard (calvin.pritchard@asu.edu) CoMSES

An Interface Definition Library for Existing Models

- support remote access to models (right now command line programs)
- help to validate, serialize and deserialize models interfaces and messages
- intended to enable wrapping existing models by inspecting metadata

Concepts

- schemas - declare the shape of source/sink data
- resources - declare the location of data or contain data themselves
- methods - have a name as well as a list of sink and source schemas to validate incoming requests 
- classes - have a name and a list of methods
- method requests - have a class name and method name for routing to the appropriate class method as well as lists of sink and source resources

![request_cycle](interaction.svg)

![Request](request.svg)

Schemas

```
table Schema {
    name: string;
    type_name: string;
    payload: [ubyte];
}
```

The schema is a payload with a name and a type_name. The type_name is a key that tells you the format of the payload. Payload is a binary buffer so that different schema formats can be added by other libraries. The name is the name of the sink or source that is associated with the validator

An example schema that loads and saves using Pandas. The schema matches [Apache Arrows](https://arrow.apache.org/docs/python/data.html#schemas).

```python
PandasHandler.from_kwargs(
    description='Daily soil characteristics',
    columns={
        'fields': [
            {
                'name': name,
                'data_type': 'Float32'
            } for name in
            [
                'day_of_year',
                'soil_daily_runoff',
                'soil_daily_infiltration',
                'soil_daily_drainage',
                'soil_evapotranspiration',
                'soil_evaporation',
                'soil_water_storage_depth',
                'soil_water_profile_ratio',
                'soil_water_deficit_stress',
                'soil_water_excess_stress'
            ]]})
```

Function Interface

A function interface definition can be created in Python by specifying how to handle and validate the sources and sinks used by the model.


```python
simplecrop_func_interface = FuncInterfaceServer(
    name = 'run',
    sources = {
        'daily': PandasHandler.from_kwargs(...),
        'yearly': PandasHandler.from_kwargs(...)
    },
    sinks = {
        'plant': PandasHandler.from_kwargs(...),
        'soil': PandasHandler.from_kwargs(...)
    })
```

A class interface definition can be created in Python by specifying different function methods.

```python
simplecrop_class_interface = ClassInterfaceServer(
    name = 'simplecrop',
    methods=[
        FuncInterfaceServer(
            name = 'default_year_config',
            sources = {},
            sinks = {
                'yearly': PandasHandler.from_kwargs(...)
            }),
        FuncInterfaceServer(
            name = 'run',
            sources = {
                'daily': PandasHandler.from_kwargs(...),
                'yearly': PandasHandler.from_kwargs(...)
            },
            sinks = {
                'plant': PandasHandler.from_kwargs(...),
                'soil': PandasHandler.from_kwargs(...)
            })
    ])
```

```python
simplecrop = ClientClassModel.from_path(
    name='simplecrop',
    class_name='simplecrop',
    path='simplecrop_omf',
    trial=trial)

sinks = simplecrop.default_year_config()
```

A request to a function endpoint (a command line program)

```python
overlandflow = ClientFunctionModel.from_path(
    name='overlandflow', 
    path=os.path.join(BASE_DIR, 'overlandflow/model.py'),
    trial=trial
)

elevation = FileResource(".asc")
weather = FeatherResource()

sources = {
    'elevation': elevation,
    'weather': weather
}

sinks = overlandflow.run(sources=sources)
```

BMI Interface methods supported

- intialize
- get_input_var_names
- get_input_var_type
- get_output_var_names
- get_output_var_type
- get_value
- set_value
- update
- finalize

BMI interface in action

```python
simplecrop_bmi = BMI(CLIRef(
    name='simplecrop', 
    class_name='simplecrop', 
    path='simplecrop_omf'))
simplecrop_bmi.initialize(trial=trial)

simplecrop_bmi.get_input_var_names()
simplecrop_bmi.get_output_var_names()

simplecrop_bmi.set_value('daily', FeatherResource())
simplecrop_bmi.set_value('yearly', FeatherResource())
simplecrop_bmi.update()
plant = simplecrop_bmi.get_value('plant')
soil = simplecrop_bmi.get_value('soil')
simplecrop_bmi.finalize()
```

# Roadmap / Future Directions

- distribute packages on PyPi for MacOS and Windows
- make resource handling easily available to other packages
- document how to build your own resource handlers
- support model communication via grpc with [Arrow Flight](https://arrow.apache.org/docs/format/Flight.html) services
- improve integration with [Prefect](https://docs.prefect.io/) (workflow manager)
- contribute changes back to BMI / [PyMT](https://pymt.readthedocs.io/en/latest/)
- support additional languages such as Java, Julia and R

# Questions for the audience

- what sort of resources would need to be supported in order meillionen to be useful to you?
- what sort of example wrappers around existing frameworks would you like to see?
- what programming languages do we need to support in order for the framework to be useful?

# Challenges: Coupled Remote Models

- How to call off to and setup remote coupled models (overlandflow is coupled with an infiltration model in the example)
- The overlandflow creates a coupled model interface manually
- Would be better to use existing model metadata to build an interface automatically (or at least partially automatically)
-  Will work on using metadata in existing BMI models in the coming weeks to reduce boilerplate needed to wrap a model

# Challenges: Model Adapters

- Name mismatches in dataframe columns and tensor dimensions between models require adapters right now
- Resources for dataframes and tensors should have optional selected fields and field aliases to remove some of the times adapter functions / classes need to be created
- More involved data transformation cases can make use of datafusion or other frameworks to provide an SQL interface so that joins, aggregates, selects and filters can be done to the data

# Challenges: Dynamic Interfaces

- Some model interfaces may you may want to change based on input from a previous function
  - Saving only particular variables in a simulation
  - Type of source constrains type of sink (keep projection of source and use it in sink type)
- Can be done with multiple methods and explicitly providing schema information 

# Resources

- [meillionen](https://github.com/openmodelingfoundation/meillionen)
- [PyMT](https://pymt.readthedocs.io/en/latest/)
- [OMF](https://openmodelingfoundation.github.io)